In [1]:
urls = ["https://www.lighting.philips.com/content/b2b-philips-lighting/vn/vi/products/highlighted-products/ledstrips",
       "https://www.lighting.philips.com/content/b2b-philips-lighting/ph/en/support/cases/airports/schiphol-airport",
        "https://www.lighting.philips.com/content/b2b-philips-lighting/ph/en/products/highlighted-products/ledstrips",
        "https://www.lighting.philips.com/content/b2b-philips-lighting/vn/vi/products/highlighted-products/maxos-fusion",
        "https://www.as.com",
        "https://www.eldiario.es/internacional/terremoto-magnitud-8-8-frente-peninsula-rusa-kamchatka-desata-alertas-tsunami-pacifico_1_12501817.html",
        "https://www.lighting.philips.com/content/b2b-philips-lighting/ph/en/application-areas/public-spaces/recreational-sports/outdoor-sports"
]

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv("C:/Users/Usuario/Documents/marketo_python/urls_pageviews.csv")
df.head(5)

,urls
0,https://www.lighting.philips.com/content/b2b-p...
1,https://www.lighting.philips.com/content/b2b-p...
2,https://www.lighting.philips.com/content/b2b-p...
3,https://www.lighting.philips.com/content/b2b-p...
4,https://www.lighting.philips.com/content/b2b-p...


In [4]:
urls = df["urls"].iloc[:5].tolist()

In [5]:
import asyncio
import nest_asyncio
import csv
from playwright.async_api import async_playwright

nest_asyncio.apply()

# Lista de URLs a revisar


async def revisar_pagina(page, url):
    print(f"\n🌐 Revisando: {url}")
    error_404_detectado = False
    munchkin_detectado = False

    async def manejar_respuesta(response):
        nonlocal error_404_detectado
        if response.status == 404:
            print(f"❌ Recurso 404 detectado: {response.url}")
            error_404_detectado = True

    def manejar_request(request):
        nonlocal munchkin_detectado
        if "munchkin.js" in request.url:
            print(f"📦 Munchkin code detectado: {request.url}")
            munchkin_detectado = True

    page.on("response", manejar_respuesta)
    page.on("request", manejar_request)

    try:
        await page.goto(url, timeout=20000)
        await page.wait_for_timeout(3000)
    except Exception as e:
        print(f"⚠️ Error al cargar la página: {e}")
        return {"url": url, "error_404_detectado": True, "munchkin_detectado": munchkin_detectado}

    return {
        "url": url,
        "error_404_detectado": error_404_detectado,
        "munchkin_detectado": munchkin_detectado
    }

async def main():
    resultados = []

    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=False)
        context = await browser.new_context()

        # Pestaña en blanco que permanece abierta
        pestana_principal = await context.new_page()
        paginas_abiertas = [pestana_principal]

        for url in urls:
            nueva_pestana = await context.new_page()
            paginas_abiertas.append(nueva_pestana)

            if len(paginas_abiertas) > 2:
                anterior = paginas_abiertas.pop(1)
                await anterior.close()

            resultado = await revisar_pagina(nueva_pestana, url)
            resultados.append(resultado)

        print("\n📝 Guardando resultados...")

        with open("resultados_munchkin_pageview29.csv", mode="w", newline="", encoding="utf-8") as f:
            writer = csv.DictWriter(f, fieldnames=["url", "error_404_detectado", "munchkin_detectado"])
            writer.writeheader()
            writer.writerows(resultados)

        print("✅ Resultados guardados en 'resultados_munchkin_pageview29.csv'")


if __name__ == "__main__":
    asyncio.run(main())



🌐 Revisando: https://www.lighting.philips.com/content/b2b-philips-lighting/my/en/application-areas/retail-hospitality/fruits-vegetables
❌ Recurso 404 detectado: https://www.lighting.philips.com.my/application-areas/retail-hospitality/ui.svg
📦 Munchkin code detectado: https://munchkin.marketo.net/munchkin.js
📦 Munchkin code detectado: https://munchkin.marketo.net/164/munchkin.js

🌐 Revisando: https://www.lighting.philips.com/content/b2b-philips-lighting/ph/en/application-areas/public-spaces/airports/air-side
📦 Munchkin code detectado: https://munchkin.marketo.net/munchkin.js
📦 Munchkin code detectado: https://munchkin.marketo.net/164/munchkin.js
❌ Recurso 404 detectado: https://www.lighting.philips.com.ph/application-areas/public-spaces/airports/ui.svg

🌐 Revisando: https://www.lighting.philips.com/content/b2b-philips-lighting/ph/en/application-areas/office-industry/office/cri90-upgrade-professional-luminaire
❌ Recurso 404 detectado: https://www.lighting.philips.com.ph/application-area